This tutorial will demonstrate how to work with XDF objects in MindPype. We will create a pipeline that performs addition to demonstrate this.

In [ ]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [ ]:
import mindpype as mp
import numpy as np

In [ ]:
channels = [i for i in range(3,10)]
tasks = ('flash', 'target')
file = ['/path/to/mindset/data']
start = -.2
samples = 500

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline. After creating the session we will create two graphs, one to test epoched data and one to test continous data.

In [ ]:
# Create a session
session = mp.Session.create()

# Create two graphs, one to test the epoched data and one to test the continuous data
graph = mp.Graph.create(session)
graph_cont = mp.Graph.create(session)

Next, we will create XDF objects for our epoched and continous data. This can be done by using the ```create_epoched()``` and ```create_continuous()``` factory methods.

In [ ]:
# Create the XDF objects for the epoched and continuous data using the factory methods
xdf_object = mp.source.BcipXDF.create_epoched(session, file, tasks, channels, start, samples)
xdf_object_continuous = mp.source.BcipXDF.create_continuous(session, file, tasks, channels, start, samples)

Now, we will create our input and output data containers for the graph. For both the epoched and the continuous case, we can use the factory method ```create_from_handle()``` to create a Tensor from the XDF sources.

For the second input to our addition kernel we can use the ```create_from_data()``` factory method to create our Tensor since the input data is known.

We will also create Tensors to store our output data for the epoched and continuous case. We can create a Tensor of a specified shape using the ```Tensor.create()``` factory method.

In [ ]:
# Create the input tensors for the epoched and continuous data
t_in = mp.Tensor.create_from_handle(session, (len(channels), samples), xdf_object)
t_in_cont = mp.Tensor.create_from_handle(session, (len(channels), samples), xdf_object_continuous)

# Create an input tensor for the second input to the addition kernel
t_in_2 = mp.Tensor.create_from_data(session, data=np.ones(t_in.shape))

# Create the output tensors for the epoched and continuous data
t_out = mp.Tensor.create(session, shape=t_in.shape)
t_out_cont = mp.Tensor.create(session, shape=t_in.shape)

Next, we will add an addition kernel to the epoched and continous graphs by using the ```add_to_graph()``` factory method.

In [ ]:
# Add the addition kernel to each graph
mp.kernels.AdditionKernel.add_to_graph(graph, t_in, t_in_2, t_out)
mp.kernels.AdditionKernel.add_to_graph(graph_cont, t_in_cont, t_in_2, t_out_cont)


At this point we have created the processing pipeline, and added all of the required components. We can now use a graph method to verify that the structure of the pipeline is valid.

Following the verification of the pipeline, we should now initialize the graph. This step is required for pipelines that have methods that need to be trained or fit (ie. classifiers), but optional for other pipelines.

In [ ]:
# Verify and initialize the graphs
graph.verify() 
graph_cont.verify()

graph.initialize() 
graph_cont.initialize()

Finally, we can run our pipeline by calling the ```execute()``` method on our graphs.

In [ ]:
# Execute the graphs
for i in range(9):
    graph.execute()
    graph_cont.execute()  

    # Check that the output tensors are equal to the sum of the input tensors
    print(np.array_equal(t_out.data, t_in.data + t_in_2.data))
    print(np.array_equal(t_out_cont.data, t_in_cont.data + t_in_2.data)) 
